In [ ]:
import os

In [ ]:
# Set JAVA_HOME to the path of Java 17
# (This command finds the path dynamically using the mac system tool)
java17_path = os.popen("/usr/libexec/java_home -v 17").read().strip()

if java17_path:
    os.environ["JAVA_HOME"] = java17_path
    print(f"Successfully set JAVA_HOME to: {java17_path}")
else:
    print("Error: Java 17 not found! Please verify installation.")

In [ ]:
print("Hello World")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.ml.feature import Imputer

In [ ]:
#!pip install pyspark

In [ ]:
spark=SparkSession.builder.appName("LearnSpark").getOrCreate()

In [ ]:
spark.range(5).show()

In [ ]:
df_pyspark=spark.read.csv('test.csv')

In [ ]:
df_pyspark.show()

In [ ]:
df_pyspark=spark.read.option('header','true').csv('test.csv',inferSchema=True)

In [ ]:
df_pyspark.head(3)

In [ ]:
df_pyspark.printSchema()

In [ ]:
df_pyspark.columns

In [ ]:
rows=df_pyspark.head(3)
for row in rows:
    print(type(row))

In [ ]:
df_pyspark.select(["studentId","secondName"]).show()

In [ ]:
df_pyspark.describe().show()

In [ ]:
df_pyspark=df_pyspark.withColumn('fullName',concat_ws(" ",df_pyspark.firstName,df_pyspark.secondName))

In [ ]:
df_pyspark.show()

In [ ]:
df_pyspark=df_pyspark.drop('fullName')

In [ ]:
df_pyspark.show()

In [ ]:
df_pyspark=df_pyspark.withColumnRenamed('studentId','Id')

In [ ]:
df_pyspark.show()

In [ ]:
df_pyspark.na.fill('Missing Value').show()

In [ ]:
imputer = Imputer(
    inputCols=['Age', 'Experience', 'Salary'], 
    outputCols=["{}_imputed".format(c) for c in ['age', 'Experience', 'Salary']]
    ).setStrategy("median")

In [ ]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

In [ ]:
df_pyspark.filter('Salary > 200000').show()

In [ ]:
df_pyspark.groupBy(['firstName','secondName']).sum().show()